In [1]:
import json
import pickle
import pandas as pd

from tensorflow.keras.models import load_model                      # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer           # type: ignore
from tensorflow.keras.preprocessing.text import tokenizer_from_json # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences   # type: ignore

#=== === === === === ===

class TheModels:

    # loading the tokenizer config and creating it
    tokenizer_config = json.load(open('saved_models/tokenizer_config.json'))
    tokenizer = tokenizer_from_json(tokenizer_config)
    print(f"Tokenizer loaded ...")

    # loading the model for scaling
    scaler = pickle.load(open('saved_models/scaler.pkl', 'rb'))
    print(f"Scaler loaded ...")

    # loading the model for prediction
    model = load_model('saved_models/model_03.h5')
    print(f"Model loaded ...")

    def __init__(self, value):
        self.instance_variable = value

#=== === === === === ===

def get_prediction(message):
    
    # pre-processing the messages
    X_text, X_features = _preprocess(message)
    
    # executing the model
    prediction = _execute_model(X_text, X_features)

    # returning the prediction
    return prediction[0][0]

#=== === === === === ===

def _preprocess(message):

    # converting message to array
    message_array = [message]
    print(f"Message array: {message_array}")

    # tokenizing the message
    X_text = TheModels.tokenizer.texts_to_sequences(message_array)
    print(f"Message tokenized: {X_text}")
    X_text = pad_sequences(X_text, maxlen=200)
    print(f"Message padded: {X_text}")

    #--- --- ---

    # getting the additional features
    df_feats    = pd.DataFrame({'sentiment_polarity': [0.3], 'sentiment_subjectivity': [0.5], 'dominant_topic': [0.7]})
    feats_array = df_feats[['sentiment_polarity', 'sentiment_subjectivity', 'dominant_topic']].values
    print(f"Features array: {feats_array}")

    #--- --- ---

    # normalizing/scaling the additional features
    X_features = TheModels.scaler.transform(feats_array)
    print(f"Features scaled: {X_features}")

    #--- --- ---

    # returning the pre-processed messages
    return X_text, X_features

#=== === === === === ===

def _execute_model(X_text, X_features):

    # executing the model and getting the prediction
    y_pred = TheModels.model.predict([X_text, X_features])
    print(f"Prediction: {y_pred}")

    # returning the prediction
    return y_pred

#=== === === === === ===

Tokenizer loaded ...
Scaler loaded ...
Model loaded ...


In [2]:
#=== === === === === === ===

# evaluating the individual message
def _evaluate_message(userFrom, message):

    # logging the message
    print(f"Evaluating message  from user {userFrom}: {message}")

    # getting and returning the prediction
    return get_prediction(message)

#=== === === === === === ===

all_messages = { }

# evaluating all messages from the user to the other user
def _evaluate_all_messages(userFrom, userTo, message):

    # getting the accumulated messages
    key_messages = f"{userFrom}||{userTo}"
    if key_messages not in all_messages:
        all_messages[key_messages] = { "messages": message + ". ", "prediction": None }
    else:
        all_messages[key_messages] = { "messages": all_messages[key_messages]["messages"] + message + ". ", "prediction": None }

    # logging the messages
    print(f"Evaluating messages from user {userFrom} to {userTo}: {all_messages[key_messages]['messages']}")
    
    # getting and keeping the prediction
    all_messages[key_messages]["prediction"] = get_prediction(all_messages[key_messages]["messages"])
    # returning the prediction
    return all_messages[key_messages]["prediction"]

#=== === === === === === ===

In [5]:
# getting the input parameters
userFrom = 'haldo'
userTo   = 'eduardo'
message  = 'hola'

# evaluating the message
resultThisMsg = _evaluate_message(userFrom, message)
# evaluating all messages from the user to the other user
resultAllMsgs = _evaluate_all_messages(userFrom, userTo, message)

# returning the results
result = { 
    'resultThisMsg': round(float(resultThisMsg),3), 
    'resultAllMsgs': round(float(resultAllMsgs),3) 
}
result

Evaluating message  from user haldo: hola
Message array: ['hola']
Message fitted: ['hola']
Message tokenized: [[]]
Message padded: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Features array: [[0.3 0.5 0.7]]
Features scaled: [[ 0.96989972  0.38361821 -1.54672122]]
1/1 [==============================] - 0s 88ms/step
Prediction: [[0.02471528]]
Evaluating messages from user haldo to eduardo: hola. hola. hola. 
Message array: ['hola. hola. hola. ']
Message fitted: ['hola. hola. hola. ']
Message tokenized: [[]]
Message padded: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0

{'resultThisMsg': 0.025, 'resultAllMsgs': 0.025}